In [1]:
from google import genai
from google.genai import types
import pathlib
import httpx
import os
import json
import sys

In [2]:
import sys
import os

# 프로젝트 루트, Common, 그리고 Flash_Card 폴더를 경로에 추가
root_path = os.path.abspath(os.path.join('..'))
common_path = os.path.abspath(os.path.join('..', 'Common'))
flash_card_gen_path = os.path.abspath(os.path.join('Generate_flashCard'))

if root_path not in sys.path:
    sys.path.append(root_path)
if common_path not in sys.path:
    sys.path.append(common_path)
if flash_card_gen_path not in sys.path:
    sys.path.append(flash_card_gen_path)

from Common.Prior_Profile_Gen_Agent import Execute_Generator_PriorProfile
from Common.generate_FeedBack import Generator_feedBack
from Generate_flashCard import Generate_flashCard

print("임포트 완료: Execute_Generator_PriorProfile, Generate_flashCard, Generator_feedBack")

임포트 완료: Execute_Generator_PriorProfile, Generate_flashCard, Generator_feedBack


In [3]:
from datetime import datetime

def Workflow_FlashCard_Generation(lecture_material, prev_userProfile=None):
    exam_type = "Flash_Card"
    
    print(f"\n{'='*20} [Workflow 시작] {'='*20}")
    
    # 1. 사전 Profile 제작
    print("\n[Step 1] 사용자 학습 프로필 생성/수정 단계")
    user_profile = Execute_Generator_PriorProfile(prev_userProfile, exam_type)
    print(">> 사용자 프로필 확정 완료.")

    # 2. 카드 개수 결정 (UI 시뮬레이션)
    print("\n[Step 2] 생성할 플래시 카드 개수 결정")
    while True:
        try:
            target_card_count = int(input("생성할 플래시 카드 개수를 입력하세요 (최대 30개, 기본 15개): ") or 15)
            if 1 <= target_card_count <= 30:
                break
            print("!! 1에서 30 사이의 숫자를 입력해주세요.")
        except ValueError:
            print("!! 올바른 숫자를 입력해주세요.")
    print(f">> 목표 카드 수: {target_card_count}개")

    # 3. 플래시 카드 생성
    print("\n[Step 3] 플래시 카드 생성 및 품질 검증 단계 시작")
    generated_cards = Generate_flashCard(
        user_profile=user_profile, 
        lecture_material=lecture_material, 
        target_card_count=target_card_count
    )
    
    # 4. 결과 출력 (UI 표현 시뮬레이션)
    print(f"\n{'='*20} [생성된 플래시 카드] {'='*20}")
    for card in generated_cards.get('flash_cards', []):
        print(f"ID: {card['id']} | [{card['category_tag']}]")
        print(f"  Q: {card['front_content']}")
        print(f"  A: {card['back_content']}")
        print("-" * 50)

    # 5. User 피드백 생성 에이전트 호출
    print("\n[Step 5] 학습 결과 로그 및 피드백 데이터 생성")
    current_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    feedback_data = Generator_feedBack(
        test_log_data=generated_cards, # flash_cards 리스트가 포함된 전체 결과 전달
        exam_type=exam_type,
        source_material=lecture_material,
        timestamp=current_timestamp
    )
    
    print(">> 피드백 데이터 생성 완료.")
    print(f"{'='*20} [Workflow 종료] {'='*20}\n")
    
    return {
        "user_profile": user_profile,
        "generated_cards": generated_cards,
        "feedback_data": feedback_data
    }



In [4]:
# 실행 예시 (주석 해제 후 사용)
lecture_material = "/Users/jhkim/Desktop/LectureTestGenerator/SamSung(260109).pdf"
result = Workflow_FlashCard_Generation(lecture_material)


==================== [Workflow 시작] ====================

[Step 1] 사용자 학습 프로필 생성/수정 단계
플래시카드로 제작하고 싶은 학습 주제나 키워드는 무엇인가요? 그리고 현재 해당 주제에 대한 본인의 숙련도(초급/중급/고급)를 알려주세요.
프로필 작성이 완료되었습니다:
=== 학습 목표 ===
  집중 학습 영역: 삼성전자의 주가 전망
  목표 이해 수준: Concept
  문제 출제 스타일: Balance

=== 사용자 상태 ===
  현재 지식 수준: Intermediate
  취약점 집중 모드: False

=== 상호작용 스타일 ===
  언어 선호: Korean_with_English_Terms
  시나리오 기반 문제: False

=== 피드백 선호 ===
  채점 엄격도: Lenient
  해설 깊이: Answer_Only

=== 지식 범위 ===
  탐색 범위: Lecture_Material_Only
이대로 진행하시겠습니까? 수정사항이 있다면 입력해주세요. (Enter to confirm)
>> 사용자 프로필 확정 완료.

[Step 2] 생성할 플래시 카드 개수 결정
>> 목표 카드 수: 5개

[Step 3] 플래시 카드 생성 및 품질 검증 단계 시작

[Step 1] 핵심 개념 추출 및 출제 계획 수립 중... (목표 카드 수: 5)
>> 출제 계획 수립 완료.

[Step 2-1] 플래시 카드 생성 및 검증 시도 (Attempt 1/3)
  - 카드 생성 중...
  - 5개의 카드 생성 완료.
  - 품질 검증 중...
>> 품질 검증 통과! 최종 결과물을 반환합니다.

==================== [생성된 플래시 카드] ====================
ID: 1 | [실적 분석]
  Q: 2025년 4분기 삼성전자 DS(Device Solutions) 부문의 영업이익이 전 분기 대비 약 10조 원가량 대폭 증가한 주요 원인은 무엇인가?
  A: DRAM 및 SSD 

In [5]:
print(result.get('generated_cards'))

{'flash_cards': [{'id': 1, 'front_content': '2025년 4분기 삼성전자 DS(Device Solutions) 부문의 영업이익이 전 분기 대비 약 10조 원가량 대폭 증가한 주요 원인은 무엇인가?', 'back_content': 'DRAM 및 SSD 수요 증가에 따른 가격 상승 효과와 HBM(High Bandwidth Memory) 물량 증가 영향', 'category_tag': '실적 분석'}, {'id': 2, 'front_content': '2025년 4분기 MX(Mobile eXperience) 사업부의 수익성이 예상치 대비 크게 부진했던 주요 원인은 무엇인가?', 'back_content': '메모리 가격 상승으로 인한 원가 구조(Cost Structure)의 악화', 'category_tag': '실적 분석'}, {'id': 3, 'front_content': '삼성전자의 목표주가 180,000원을 산출하기 위해 2026년 예상 BPS(주당순자산가치) 78,295원에 적용한 PBR(주가순자산비율) 배수는 얼마인가?', 'back_content': '2.3배', 'category_tag': 'Valuation'}, {'id': 4, 'front_content': '리포트에서 DRAM과 NAND의 ASP(Average Selling Price, 평균판매단가) 상승폭이 이전 전망 대비 컸다고 추정한 것은 실적에 어떤 의미를 갖는가?', 'back_content': '판매 단가 상승을 통해 부품 부문의 수익성 방어 및 이익 개선에 기여함', 'category_tag': '수익성 지표'}, {'id': 5, 'front_content': '삼성전자의 향후 주가 강세가 지속될 것으로 전망하는 근거 중, 차세대 메모리 시장과 관련된 기대 요소는 무엇인가?', 'back_content': 'AI 투자 확대에 따른 메모리 수요 본격화 및 2026년 HBM4 시장에서의 의미 있는 점유율 확보 전망', 'category_tag': '주